# Packages

In [171]:
from geopy.geocoders import Nominatim
import geopy
geolocator = Nominatim(user_agent="geoapiExercises")
import undetected_chromedriver as uc
from bs4 import BeautifulSoup
import requests
import numpy as np
import random
import math
import time as t
import pandas as pd
import re
from deep_translator import GoogleTranslator
from tqdm.notebook import tqdm
from fake_useragent import UserAgent
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.common.keys import Keys

# Function

In [2]:
def flatten(A):
    rt = []
    for i in A:
        if isinstance(i,list): rt.extend(flatten(i))
        else: rt.append(i)
    return rt

In [94]:
def sqftosqmt(num):
    return num/10.764

In [4]:
def has_number(input_string):
    pattern = r"\d+"
    match = re.search(pattern, input_string)
    if match:
        return True
    else:
        return False

In [5]:
def extract_number(input_string):
    pattern = r"(\d+)"
    match = re.search(pattern, input_string)
    if match:
        extracted_number = int(match.group(1))  # Convert the matched number to an integer
        return extracted_number
    else:
        return None 

In [6]:
def addressToCoord(address):
    loc = Nominatim(user_agent="Geopy Library")
    getLoc = loc.geocode(address)
    return(getLoc.latitude, getLoc.longitude)

In [7]:
def is_string_in_another_string_once(main_string, sub_string):
    index = main_string.find(sub_string)
    if index != -1:
        second_index = main_string.find(sub_string, index + 1)
        if second_index == -1:
            return True
    return False

In [8]:
def uniquize(seq):
    seen = set()
    seen_add = seen.add
    return [x for x in seq if not (x in seen or seen_add(x))]

In [9]:
def contains_only_numbers(input_string):
    pattern = r"^\d+$"
    return bool(re.match(pattern, input_string))

In [10]:
ua = UserAgent()

# Scrape

In [11]:
mother_url = 'https://www.engelvoelkers.com/de'
apartments = 'https://www.engelvoelkers.com/de/search/?q=&startIndex=0&businessArea=&sortOrder=DESC&sortField=newestProfileCreationTimestamp&pageSize=18&facets=cntry%3Agermany%3Bobjcttyp%3Acondo%3Btyp%3Arent%3B'
houses = 'https://www.engelvoelkers.com/de/search/?q=&startIndex=0&businessArea=&sortOrder=DESC&sortField=newestProfileCreationTimestamp&pageSize=18&facets=cntry%3Agermany%3Bobjcttyp%3Ahouse%3Btyp%3Arent%3B'

In [72]:
apartments = 'https://www.engelvoelkers.com/de/search/?q=&startIndex=0&businessArea=&sortOrder=DESC&sortField=newestProfileCreationTimestamp&pageSize=18&facets=cntry%3Agermany%3Bobjcttyp%3Acondo%3Btyp%3Arent%3B'
houses = 'https://www.engelvoelkers.com/de/search/?q=&startIndex=0&businessArea=&sortOrder=DESC&sortField=newestProfileCreationTimestamp&pageSize=18&facets=cntry%3Agermany%3Bobjcttyp%3Ahouse%3Btyp%3Arent%3B'

## Pagination Generation

In [73]:
header = {'User-Agent':str(ua.random)}
content = requests.get(apartments, headers = header)
soup = BeautifulSoup(content.content, 'html.parser')

pages_container = soup.find('ul', {'class':'ev-pager-row'})
pages_container_narrow = pages_container.find_all('li', {'class':'ev-pager-item'})[-2]
last_page = int(pages_container_narrow.text.strip())
pages_apartments = [apartments.replace('startIndex=0', 'startIndex={}'.format(str(i*18))) for i in range(0, last_page)]

In [74]:
header = {'User-Agent':str(ua.random)}
content = requests.get(houses, headers = header)
soup = BeautifulSoup(content.content, 'html.parser')

pages_container = soup.find('ul', {'class':'ev-pager-row'})
pages_container_narrow = pages_container.find_all('li', {'class':'ev-pager-item'})[-2]
last_page = int(pages_container_narrow.text.strip())
pages_houses = [houses.replace('startIndex=0', 'startIndex={}'.format(str(i*18))) for i in range(0, last_page)]

## Product Links

In [84]:
all_products_apartments = []

for page in tqdm(pages_apartments):
    header = {'User-Agent':str(ua.random)}
    content = requests.get(page, headers = header)
    soup = BeautifulSoup(content.content, 'html.parser')

    product_container = soup.find('div', {'class':'row ev-search-results'})
    if product_container:
        product_container_narrow = product_container.find_all('div', {'class':'col-lg-4 col-md-4 col-sm-6 col-xs-12'})

        products = []
        for i in product_container_narrow:
            try:
                if 'exposes' in i.find('a', {'href':True})['href']:
                    products.append(i.find('a', {'href':True})['href'])
                else:
                    continue
            except:
                continue

        all_products_apartments.append(products)
    else:
        continue
        
all_products_apartments = uniquize(flatten(all_products_apartments))

In [90]:
all_products_houses = []

for page in tqdm(pages_houses):
    header = {'User-Agent':str(ua.random)}
    content = requests.get(page, headers = header)
    soup = BeautifulSoup(content.content, 'html.parser')

    product_container = soup.find('div', {'class':'row ev-search-results'})
    
    if product_container:
        product_container_narrow = product_container.find_all('div', {'class':'col-lg-4 col-md-4 col-sm-6 col-xs-12'})

        products = []
        for i in product_container_narrow:
            try:
                if 'exposes' in i.find('a', {'href':True})['href']:
                    products.append(i.find('a', {'href':True})['href'])
                else:
                    continue
            except:
                continue

        all_products_houses.append(products)
    else:
        continue
    
all_products_houses = uniquize(flatten(all_products_houses))

## Data Collection

### Apartments

In [209]:
all_sizes = []
all_links1 = []
all_rooms1 = []
all_final_costs1 = []
all_zip_codes1 = []
all_latitudes1 = []
all_longitudes1 = []
all_balconies = []
all_storages = []
all_elevators = []
all_years1 = []


for link in tqdm(all_products_apartments):
    header = {'User-Agent':str(ua.random)}
    content = requests.get(link, headers = header)
    soup = BeautifulSoup(content.content, 'html.parser')
    
    test_price = soup.find('div', {'class':'ev-exposee-content ev-key-facts'})
    if test_price:
        price_container = test_price.find('span', {'itemprop':'price'})
        price = price_container.text.strip().replace('€', '').replace('.', '').replace(',', '.')
        if has_number(price):
            price = float(price.replace('€', '').replace('.', '').replace(',', '.').replace('  VB', ''))
        else:
            price = None
    else:
        price = None

    test_address = soup.find('div', {'class':'ev-exposee-content ev-exposee-subtitle'})
    if test_address:
        address = test_address.text.split('|')[-1].strip(' ')

        latitude = None
        longitude = None
        zip_code = None
        try:
            latitude = addressToCoord(address)[0]
            longitude = addressToCoord(address)[1]
        except:
            latitude = None
            longitude = None

        if latitude: 
            try:
                geo_locator = geopy.Nominatim(user_agent='1234')
                r = geo_locator.reverse(addressToCoord(address))
                zip_code = r.raw['address']['postcode']
            except:
                zip_code = None
        else:
            zip_code = None

    test_additionals = soup.find_all('li', {'class':'ev-exposee-detail-fact'})
    if test_additionals:
        additionals_raw = [i.text.strip(' ').lower() for i in test_additionals]

        interior_size = None
        rooms = None
        year = None
        balcony = 0
        storage = 0
        parking = 0
        elevator = 0
        for component in additionals_raw:
            if 'wohnfläche' in component and 'gesamt' not in component:
                interior_size = float(component.replace('wohnfläche ', '').replace('ca. ', '').replace(' m²', '').replace('.', '').replace(',', '.'))
            elif 'zimmer' in component and 'bade' not in component and 'schlaf' not in component and 'ess' not in component and 'privat' not in component:
                rooms = float(component.replace('zimmer ', '').replace('.', '').replace(',', '.'))
            elif 'baujahr' in component:
                year = int(component.replace('baujahr ', ''))
            elif 'balkon' in component or 'terrasse':
                balcony = 1
            elif 'keller' in component:
                storage = 1
            elif 'stellplatz' in component or 'garage' in component or 'stellplätze' in component:
                parking = 1
            elif 'aufzug' in component:
                elevator = 1
            else:
                continue
    else:
        interior_size = None
        rooms = None
        year = None
        balcony = 0
        storage = 0
        parking = 0
        elevator = 0    
                
    all_sizes.append(interior_size)
    all_rooms1.append(rooms)
    all_final_costs1.append(price)
    all_zip_codes1.append(zip_code)
    all_latitudes1.append(latitude)
    all_longitudes1.append(longitude)
    all_balconies.append(balcony)
    all_storages.append(storage)
    all_elevators.append(elevator)
    all_links1.append(link)
    all_years1.append(year)
    
all_sizes = flatten(all_sizes)
all_rooms1 = flatten(all_rooms1)
all_final_costs1 = flatten(all_final_costs1)
all_zip_codes1 = flatten(all_zip_codes1)
all_latitudes1 = flatten(all_latitudes1)
all_longitudes1 = flatten(all_longitudes1)
all_balconies = flatten(all_balconies)
all_storages = flatten(all_storages)
all_elevators = flatten(all_elevators)
all_links1 = flatten(all_links1)
all_years1 = flatten(all_years1)

### Houses

In [161]:
all_interior_sizes = []
all_exterior_sizes = []
all_house_types = []
all_links2 = []
all_rooms2 = []
all_final_costs2 = []
all_zip_codes2 = []
all_latitudes2 = []
all_longitudes2 = []
all_terraces = []
all_parkings = []
all_years2 = []

for link in tqdm(all_products_houses):
    header = {'User-Agent':str(ua.random)}
    content = requests.get(link, headers = header)
    soup = BeautifulSoup(content.content, 'html.parser')
    
    test_price = soup.find('div', {'class':'ev-exposee-content ev-key-facts'})
    if test_price:
        price_container = test_price.find('span', {'itemprop':'price'})
        price = price_container.text.strip().replace('€', '').replace('.', '').replace(',', '.')
        if has_number(price):
            price = float(price.replace('€', '').replace('.', '').replace(',', '.').replace('  VB', ''))
        else:
            price = None
    else:
        price = None

    test_address = soup.find('div', {'class':'ev-exposee-content ev-exposee-subtitle'})
    if test_address:
        address = test_address.text.split('|')[-1].strip(' ')

        latitude = None
        longitude = None
        zip_code = None
        try:
            latitude = addressToCoord(address)[0]
            longitude = addressToCoord(address)[1]
        except:
            latitude = None
            longitude = None

        if latitude: 
            try:
                geo_locator = geopy.Nominatim(user_agent='1234')
                r = geo_locator.reverse(addressToCoord(address))
                zip_code = r.raw['address']['postcode']
            except:
                zip_code = None
        else:
            zip_code = None

    test_additionals = soup.find_all('li', {'class':'ev-exposee-detail-fact'})
    if test_additionals:
        additionals_raw = [i.text.strip(' ').lower() for i in test_additionals]

        interior_size = None
        exterior_size = None
        rooms = None
        year = None
        house_type = None
        terrace = 0
        parking = 0
        for component in additionals_raw:
            if 'wohnfläche' in component and 'gesamt' not in component:
                interior_size = float(component.replace('wohnfläche ', '').replace('ca. ', '').replace(' m²', '').replace('.', '').replace(',', '.'))
            elif 'grundstück' in component:
                if 'ha' in component.replace('grundstück ', '').replace('ca. ', '').replace(' m²', '').replace('.', '').replace(',', '.'):
                    exterior_size = 10000 * float(component.replace('grundstück ', '').replace('ca. ', '').replace(' m²', '').replace('.', '').replace(',', '.').replace(' ha', ''))
                else:
                    exterior_size = float(component.replace('grundstück ', '').replace('ca. ', '').replace(' m²', '').replace('.', '').replace(',', '.'))    
            elif 'zimmer' in component and 'bade' not in component and 'schlaf' not in component and 'ess' not in component and 'privat' not in component:
                rooms = float(component.replace('zimmer ', '').replace('.', '').replace(',', '.'))
            elif 'baujahr' in component:
                year = int(component.replace('baujahr ', ''))
            elif 'balkon' in component or 'terrasse' in component:
                terrace = 1
            elif 'stellplatz' in component or 'garage' in component or 'stellplätze' in component:
                parking = 1
            elif 'aufzug' in component:
                elevator = 1
            elif 'objektunterart' in component:
                house_type = GoogleTranslator(source='de', target='en').translate(component.replace('objektunterart ', '')).lower()
            else:
                continue
    else:
        interior_size = None
        exterior_size = None
        rooms = None
        year = None
        house_type = None
        terrace = 0
        parking = 0
                
    all_interior_sizes.append(interior_size)
    all_exterior_sizes.append(exterior_size)
    all_rooms2.append(rooms)
    all_final_costs2.append(price)
    all_zip_codes2.append(zip_code)
    all_latitudes2.append(latitude)
    all_longitudes2.append(longitude)
    all_terraces.append(terrace)
    all_links2.append(link)
    all_years2.append(year)
    all_house_types.append(house_type)
    all_parkings.append(parking)

    
all_interior_sizes = flatten(all_interior_sizes)
all_exterior_sizes = flatten(all_exterior_sizes)
all_rooms2 = flatten(all_rooms2)
all_final_costs2 = flatten(all_final_costs2)
all_zip_codes2 = flatten(all_zip_codes2)
all_latitudes2 = flatten(all_latitudes2)
all_longitudes2 = flatten(all_longitudes2)
all_terraces = flatten(all_terraces)
all_links2 = flatten(all_links2)
all_years2 = flatten(all_years2)
all_house_types = flatten(all_house_types)
all_parkings = flatten(all_parkings)


# Storage

In [211]:
data_apartments = pd.DataFrame()
data_houses = pd.DataFrame()

data_apartments['Link'] = all_links1
data_apartments['Year Constructed'] = all_years1
data_apartments['Interior Size'] = all_sizes
data_apartments['Room Count'] = all_rooms1
data_apartments['Zip Code'] = all_zip_codes1
data_apartments['Latitude'] = all_latitudes1
data_apartments['Longitude'] = all_longitudes1
data_apartments['Balcony'] = all_balconies
data_apartments['Storage'] = all_storages
data_apartments['Elevator'] = all_elevators
data_apartments['Price'] = all_final_costs1
data_apartments['Zip Code'] = data_apartments['Zip Code'].fillna(0).astype(int).replace(0, None)
data_apartments['Year Constructed'] = data_apartments['Year Constructed'].fillna(0).astype(int).replace(0, None)

data_houses['Link'] = all_links2
data_houses['House Type'] = all_house_types
data_houses['Year Constructed'] = all_years2
data_houses['Interior Size'] = all_interior_sizes
data_houses['Exterior Size'] = all_exterior_sizes
data_houses['Room Count'] = all_rooms2
data_houses['Zip Code'] = all_zip_codes2
data_houses['Latitude'] = all_latitudes2
data_houses['Longitude'] = all_longitudes2
data_houses['Parking'] = all_parkings
data_houses['Terrace'] = all_terraces
data_houses['Price'] = all_final_costs2
data_houses['Zip Code'] = data_houses['Zip Code'].fillna(0).astype(int).replace(0, np.nan).replace(np.nan, None)
data_houses['Year Constructed'] = data_houses['Year Constructed'].replace(np.nan, 0).astype(int).replace(0, None)

In [212]:
def format_value(value):
    if value >= 1000 and value <= 9999:
        return f'0{value:04}'
    return str(value)

data_apartments['Zip Code'] = data_apartments['Zip Code'].apply(format_value)
data_houses['Zip Code'] = data_houses['Zip Code'].apply(format_value)

In [213]:
data_apartments.to_excel('engelVolkerApartments.xlsx', index=False, engine='openpyxl')
data_houses.to_excel('engelVolkerHouses.xlsx', index=False, engine='openpyxl')

In [67]:
data_apartments = pd.read_excel('engelVolkerApartments.xlsx', engine='openpyxl', dtype={'Zip Code': str})
data_houses = pd.read_excel('engelVolkerHouses.xlsx', engine='openpyxl', dtype={'Zip Code': str})